In [23]:
import openmeteo_requests
import plotly.express as px


import requests_cache
import pandas as pd
from retry_requests import retry

In [24]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 51.50,
	"longitude": -0.23,
	"start_date":"1941-01-01",
	"end_date": "2024-12-27",
	"hourly": "rain",
	"daily": ["weather_code", "temperature_2m_max", "temperature_2m_min", "temperature_2m_mean", "precipitation_sum", "rain_sum", "snowfall_sum", "precipitation_hours"]
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")



# Process daily data. The order of variables needs to be the same as requested.
daily = response.Daily()
daily_weather_code = daily.Variables(0).ValuesAsNumpy()
daily_temperature_2m_max = daily.Variables(1).ValuesAsNumpy()
daily_temperature_2m_min = daily.Variables(2).ValuesAsNumpy()
daily_temperature_2m_mean = daily.Variables(3).ValuesAsNumpy()
daily_precipitation_sum = daily.Variables(4).ValuesAsNumpy()
daily_rain_sum = daily.Variables(5).ValuesAsNumpy()
daily_snowfall_sum = daily.Variables(6).ValuesAsNumpy()
daily_precipitation_hours = daily.Variables(7).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s", utc = False),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = False),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}
daily_data["weather_code"] = daily_weather_code
daily_data["temperature_2m_max"] = daily_temperature_2m_max
daily_data["temperature_2m_min"] = daily_temperature_2m_min
daily_data["temperature_2m_mean"] = daily_temperature_2m_mean
daily_data["precipitation_sum"] = daily_precipitation_sum
daily_data["rain_sum"] = daily_rain_sum
daily_data["snowfall_sum"] = daily_snowfall_sum
daily_data["precipitation_hours"] = daily_precipitation_hours


daily_dataframe = pd.DataFrame(data = daily_data)

# Ensure the 'date' column is in datetime format
daily_dataframe['date'] = pd.to_datetime(daily_dataframe['date'])

daily_dataframe.info
daily_dataframe.head()

Coordinates 51.49384689331055°N -0.16302490234375°E
Elevation 7.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,weather_code,temperature_2m_max,temperature_2m_min,temperature_2m_mean,precipitation_sum,rain_sum,snowfall_sum,precipitation_hours
0,1941-01-01,3.0,0.866,-1.684,-0.700667,0.0,0.0,0.00,0.0
1,1941-01-02,2.0,-1.184,-3.534,-2.396500,0.0,0.0,0.00,0.0
2,1941-01-03,3.0,-0.684,-2.734,-1.736084,0.0,0.0,0.00,0.0
3,1941-01-04,3.0,-0.934,-2.434,-1.740250,0.0,0.0,0.00,0.0
4,1941-01-05,71.0,0.116,-2.484,-1.256917,0.4,0.0,0.28,4.0


In [25]:
# Define start and end dates for the range
start_date = '1980-01-01'
end_date = '1981-01-01'

# Filter the DataFrame for the date range
filtered_df = daily_dataframe[
    (daily_dataframe['date'] >= start_date) & (daily_dataframe['date'] < end_date)
]

filtered_df

,date,weather_code,temperature_2m_max,temperature_2m_min,temperature_2m_mean,precipitation_sum,rain_sum,snowfall_sum,precipitation_hours
14244,1980-01-01,3.0,1.304,-3.796,-1.746000,0.000000,0.000000,0.00,0.0
14245,1980-01-02,3.0,1.104,-4.396,-2.300166,0.000000,0.000000,0.00,0.0
14246,1980-01-03,71.0,9.054,-4.446,2.283167,7.199999,7.199999,0.14,15.0
14247,1980-01-04,3.0,10.004,3.104,7.387335,0.000000,0.000000,0.00,0.0
14248,1980-01-05,3.0,7.204,2.104,3.926917,0.000000,0.000000,0.00,0.0
...,...,...,...,...,...,...,...,...,...
14605,1980-12-27,51.0,3.554,-0.546,1.212333,0.100000,0.100000,0.00,1.0
14606,1980-12-28,3.0,7.004,-0.996,3.668583,0.000000,0.000000,0.00,0.0
14607,1980-12-29,3.0,8.704,6.304,7.564417,0.000000,0.000000,0.00,0.0
14608,1980-12-30,3.0,9.354,7.654,8.233167,0.000000,0.000000,0.00,0.0


In [26]:
import plotly.express as px

df = px.data.gapminder().query("country=='Canada'")
fig = px.line(daily_dataframe, x="date", y="temperature_2m_mean", title='Mean temperature from 1980 - 2024')
fig.show()

In [27]:
# Define the months you want to filter for the method .dt.moth.isin(filter_month)
winter_months = [12, 1, 2]
spring_months = [3, 4, 5]
summer_months = [6, 7, 8]
autumn_months = [9, 10, 11]

# four dataframe for the four season for comparison to compare over the years
winter_df = daily_dataframe[daily_dataframe['date'].dt.month.isin(winter_months)]
spring_df = daily_dataframe[daily_dataframe['date'].dt.month.isin(spring_months)]
summer_df = daily_dataframe[daily_dataframe['date'].dt.month.isin(summer_months)]
autumn_df = daily_dataframe[daily_dataframe['date'].dt.month.isin(autumn_months)]

# Checking the column name for filtering the data within the df
print(daily_dataframe.columns)


Index(['date', 'weather_code', 'temperature_2m_max', 'temperature_2m_min',
       'temperature_2m_mean', 'precipitation_sum', 'rain_sum', 'snowfall_sum',
       'precipitation_hours'],
      dtype='object')


In [28]:
# Creating multiply dataframe with filter and storing in a dict. The parameters selelected are decades from 1950s to 2020s within WINTER months 
df_winter = {
    "1950":daily_dataframe[daily_dataframe['date'].dt.month.isin(winter_months) & (daily_dataframe['date'].dt.year >= 1950) & (daily_dataframe['date'].dt.year <= 1959)],
    "1960":daily_dataframe[daily_dataframe['date'].dt.month.isin(winter_months) & (daily_dataframe['date'].dt.year >= 1960) & (daily_dataframe['date'].dt.year <= 1969)],
    "1970":daily_dataframe[daily_dataframe['date'].dt.month.isin(winter_months) & (daily_dataframe['date'].dt.year >= 1970) & (daily_dataframe['date'].dt.year <= 1979)],
    "1980":daily_dataframe[daily_dataframe['date'].dt.month.isin(winter_months) & (daily_dataframe['date'].dt.year >= 1980) & (daily_dataframe['date'].dt.year <= 1989)],
    "1990":daily_dataframe[daily_dataframe['date'].dt.month.isin(winter_months) & (daily_dataframe['date'].dt.year >= 1990) & (daily_dataframe['date'].dt.year <= 1999)],
    "2000":daily_dataframe[daily_dataframe['date'].dt.month.isin(winter_months) & (daily_dataframe['date'].dt.year >= 2000) & (daily_dataframe['date'].dt.year <= 2009)],
    "2010":daily_dataframe[daily_dataframe['date'].dt.month.isin(winter_months) & (daily_dataframe['date'].dt.year >= 2010) & (daily_dataframe['date'].dt.year <= 2019)],
    "2020":daily_dataframe[daily_dataframe['date'].dt.month.isin(winter_months) & (daily_dataframe['date'].dt.year >= 2020) & (daily_dataframe['date'].dt.year <= 2029)]
}

# Creating multiply dataframe with filter and storing in a dict. The parameters selelected are decades from 1950s to 2020s within SPRING months 
df_spring = {
    "1950":daily_dataframe[daily_dataframe['date'].dt.month.isin(spring_months) & (daily_dataframe['date'].dt.year >= 1950) & (daily_dataframe['date'].dt.year <= 1959)],
    "1960":daily_dataframe[daily_dataframe['date'].dt.month.isin(spring_months) & (daily_dataframe['date'].dt.year >= 1960) & (daily_dataframe['date'].dt.year <= 1969)],
    "1970":daily_dataframe[daily_dataframe['date'].dt.month.isin(spring_months) & (daily_dataframe['date'].dt.year >= 1970) & (daily_dataframe['date'].dt.year <= 1979)],
    "1980":daily_dataframe[daily_dataframe['date'].dt.month.isin(spring_months) & (daily_dataframe['date'].dt.year >= 1980) & (daily_dataframe['date'].dt.year <= 1989)],
    "1990":daily_dataframe[daily_dataframe['date'].dt.month.isin(spring_months) & (daily_dataframe['date'].dt.year >= 1990) & (daily_dataframe['date'].dt.year <= 1999)],
    "2000":daily_dataframe[daily_dataframe['date'].dt.month.isin(spring_months) & (daily_dataframe['date'].dt.year >= 2000) & (daily_dataframe['date'].dt.year <= 2009)],
    "2010":daily_dataframe[daily_dataframe['date'].dt.month.isin(spring_months) & (daily_dataframe['date'].dt.year >= 2010) & (daily_dataframe['date'].dt.year <= 2019)],
    "2020":daily_dataframe[daily_dataframe['date'].dt.month.isin(spring_months) & (daily_dataframe['date'].dt.year >= 2020) & (daily_dataframe['date'].dt.year <= 2029)]
}

# Creating multiply dataframe with filter and storing in a dict. The parameters selelected are decades from 1950s to 2020s within SUMMER months 
df_summer = {
    "1950":daily_dataframe[daily_dataframe['date'].dt.month.isin(summer_months) & (daily_dataframe['date'].dt.year >= 1950) & (daily_dataframe['date'].dt.year <= 1959)],
    "1960":daily_dataframe[daily_dataframe['date'].dt.month.isin(summer_months) & (daily_dataframe['date'].dt.year >= 1960) & (daily_dataframe['date'].dt.year <= 1969)],
    "1970":daily_dataframe[daily_dataframe['date'].dt.month.isin(summer_months) & (daily_dataframe['date'].dt.year >= 1970) & (daily_dataframe['date'].dt.year <= 1979)],
    "1980":daily_dataframe[daily_dataframe['date'].dt.month.isin(summer_months) & (daily_dataframe['date'].dt.year >= 1980) & (daily_dataframe['date'].dt.year <= 1989)],
    "1990":daily_dataframe[daily_dataframe['date'].dt.month.isin(summer_months) & (daily_dataframe['date'].dt.year >= 1990) & (daily_dataframe['date'].dt.year <= 1999)],
    "2000":daily_dataframe[daily_dataframe['date'].dt.month.isin(summer_months) & (daily_dataframe['date'].dt.year >= 2000) & (daily_dataframe['date'].dt.year <= 2009)],
    "2010":daily_dataframe[daily_dataframe['date'].dt.month.isin(summer_months) & (daily_dataframe['date'].dt.year >= 2010) & (daily_dataframe['date'].dt.year <= 2019)],
    "2020":daily_dataframe[daily_dataframe['date'].dt.month.isin(summer_months) & (daily_dataframe['date'].dt.year >= 2020) & (daily_dataframe['date'].dt.year <= 2029)]
}

# Creating multiply dataframe with filter and storing in a dict. The parameters selelected are decades from 1950s to 2020s within AUTUMN months 
df_autumn = {
    "1950":daily_dataframe[daily_dataframe['date'].dt.month.isin(autumn_months) & (daily_dataframe['date'].dt.year >= 1950) & (daily_dataframe['date'].dt.year <= 1959)],
    "1960":daily_dataframe[daily_dataframe['date'].dt.month.isin(autumn_months) & (daily_dataframe['date'].dt.year >= 1960) & (daily_dataframe['date'].dt.year <= 1969)],
    "1970":daily_dataframe[daily_dataframe['date'].dt.month.isin(autumn_months) & (daily_dataframe['date'].dt.year >= 1970) & (daily_dataframe['date'].dt.year <= 1979)],
    "1980":daily_dataframe[daily_dataframe['date'].dt.month.isin(autumn_months) & (daily_dataframe['date'].dt.year >= 1980) & (daily_dataframe['date'].dt.year <= 1989)],
    "1990":daily_dataframe[daily_dataframe['date'].dt.month.isin(autumn_months) & (daily_dataframe['date'].dt.year >= 1990) & (daily_dataframe['date'].dt.year <= 1999)],
    "2000":daily_dataframe[daily_dataframe['date'].dt.month.isin(autumn_months) & (daily_dataframe['date'].dt.year >= 2000) & (daily_dataframe['date'].dt.year <= 2009)],
    "2010":daily_dataframe[daily_dataframe['date'].dt.month.isin(autumn_months) & (daily_dataframe['date'].dt.year >= 2010) & (daily_dataframe['date'].dt.year <= 2019)],
    "2020":daily_dataframe[daily_dataframe['date'].dt.month.isin(autumn_months) & (daily_dataframe['date'].dt.year >= 2020) & (daily_dataframe['date'].dt.year <= 2029)]
}

# Checking the amount of days that is found within each season
sum_winter_2010s_total = df_winter['2010']['date'].count()
sum_spring_2010s_total = df_spring['2010']['date'].count()
sum_summer_2010s_total = df_summer['2010']['date'].count()
sum_autumn_2010s_total = df_autumn['2010']['date'].count()

sum_winter_1990s_total = df_winter['1990']['date'].count()
sum_spring_1990s_total = df_spring['1990']['date'].count()
sum_summer_1990s_total = df_summer['1990']['date'].count()
sum_autumn_1990s_total = df_autumn['1990']['date'].count()

sum_winter_1960s_total = df_winter['1960']['date'].count()
sum_spring_1960s_total = df_spring['1960']['date'].count()
sum_summer_1960s_total = df_summer['1960']['date'].count()
sum_autumn_1960s_total = df_autumn['1960']['date'].count()


print(f"Winter(2010s): {sum_winter_2010s_total}\nSpring(2010s): {sum_spring_2010s_total}\nSummer(2010s): {sum_summer_2010s_total}\nAutumn(2010s): {sum_autumn_2010s_total}\n ")
print(f"Winter(1990s): {sum_winter_1990s_total}\nSpring(1990s): {sum_spring_1990s_total}\nSummer(1990s): {sum_summer_1990s_total}\nAutumn(1990s): {sum_autumn_1990s_total}\n ")
print(f"Winter(1960s): {sum_winter_1960s_total}\nSpring(1960s): {sum_spring_1960s_total}\nSummer(1960s): {sum_summer_1960s_total}\nAutumn(1960s): {sum_autumn_1960s_total}\n ")

Winter(2010s): 902
Spring(2010s): 920
Summer(2010s): 920
Autumn(2010s): 910
 
Winter(1990s): 902
Spring(1990s): 920
Summer(1990s): 920
Autumn(1990s): 910
 
Winter(1960s): 903
Spring(1960s): 920
Summer(1960s): 920
Autumn(1960s): 910
 


In [29]:
min_winter_temp = {
    "1950":df_winter['1950']['temperature_2m_min'].mean(),
    "1960":df_winter['1960']['temperature_2m_min'].mean(),
    "1970":df_winter['1970']['temperature_2m_min'].mean(),
    "1980":df_winter['1980']['temperature_2m_min'].mean(),
    "1990":df_winter['1990']['temperature_2m_min'].mean(),
    "2000":df_winter['2000']['temperature_2m_min'].mean(),
    "2010":df_winter['2010']['temperature_2m_min'].mean(),
    "2020":df_winter['2020']['temperature_2m_min'].mean()
}

max_winter_temp = {
    "1950":df_winter['1950']['temperature_2m_max'].mean(),
    "1960":df_winter['1960']['temperature_2m_max'].mean(),
    "1970":df_winter['1970']['temperature_2m_max'].mean(),
    "1980":df_winter['1980']['temperature_2m_max'].mean(),
    "1990":df_winter['1990']['temperature_2m_max'].mean(),
    "2000":df_winter['2000']['temperature_2m_max'].mean(),
    "2010":df_winter['2010']['temperature_2m_max'].mean(),
    "2020":df_winter['2020']['temperature_2m_max'].mean()
}

mean_winter_temp = {
    "1950":df_winter['1950']['temperature_2m_mean'].mean(),
    "1960":df_winter['1960']['temperature_2m_mean'].mean(),
    "1970":df_winter['1970']['temperature_2m_mean'].mean(),
    "1980":df_winter['1980']['temperature_2m_mean'].mean(),
    "1990":df_winter['1990']['temperature_2m_mean'].mean(),
    "2000":df_winter['2000']['temperature_2m_mean'].mean(),
    "2010":df_winter['2010']['temperature_2m_mean'].mean(),
    "2020":df_winter['2020']['temperature_2m_mean'].mean()
}

# Creating a new dataframe that for the season winter that contains the "minumum","max","mean" temperature and tagging them with each season
df_winter_summary = pd.DataFrame({
    "Year": list(min_winter_temp.keys()),
    "Min Temp": list(min_winter_temp.values()),
    "Max Temp": list(max_winter_temp.values()),
    "Mean Temp": list(mean_winter_temp.values()),
    "Season": ["Winter"] * len(min_winter_temp)  # Add a tag
})

print(df_winter_summary)

   Year  Min Temp  Max Temp  Mean Temp  Season
0  1950  1.229721  6.662537   3.996346  Winter
1  1960  0.728529  6.038054   3.457251  Winter
2  1970  1.994854  7.085874   4.568195  Winter
3  1980  1.651785  6.920889   4.316592  Winter
4  1990  2.393800  7.630885   5.031880  Winter
5  2000  2.462029  7.706381   5.146223  Winter
6  2010  2.293418  7.796079   5.080361  Winter
7  2020  3.251104  9.004683   6.161126  Winter


In [30]:
min_spring_temp = {
    "1950":df_spring['1950']['temperature_2m_min'].mean(),
    "1960":df_spring['1960']['temperature_2m_min'].mean(),
    "1970":df_spring['1970']['temperature_2m_min'].mean(),
    "1980":df_spring['1980']['temperature_2m_min'].mean(),
    "1990":df_spring['1990']['temperature_2m_min'].mean(),
    "2000":df_spring['2000']['temperature_2m_min'].mean(),
    "2010":df_spring['2010']['temperature_2m_min'].mean(),
    "2020":df_spring['2020']['temperature_2m_min'].mean()
}

max_spring_temp = {
    "1950":df_spring['1950']['temperature_2m_max'].mean(),
    "1960":df_spring['1960']['temperature_2m_max'].mean(),
    "1970":df_spring['1970']['temperature_2m_max'].mean(),
    "1980":df_spring['1980']['temperature_2m_max'].mean(),
    "1990":df_spring['1990']['temperature_2m_max'].mean(),
    "2000":df_spring['2000']['temperature_2m_max'].mean(),
    "2010":df_spring['2010']['temperature_2m_max'].mean(),
    "2020":df_spring['2020']['temperature_2m_max'].mean()
}

mean_spring_temp = {
    "1950":df_spring['1950']['temperature_2m_mean'].mean(),
    "1960":df_spring['1960']['temperature_2m_mean'].mean(),
    "1970":df_spring['1970']['temperature_2m_mean'].mean(),
    "1980":df_spring['1980']['temperature_2m_mean'].mean(),
    "1990":df_spring['1990']['temperature_2m_mean'].mean(),
    "2000":df_spring['2000']['temperature_2m_mean'].mean(),
    "2010":df_spring['2010']['temperature_2m_mean'].mean(),
    "2020":df_spring['2020']['temperature_2m_mean'].mean()
}

# Combine into a DataFrame
df_spring_summary = pd.DataFrame({
    "Year": list(min_spring_temp.keys()),
    "Min Temp": list(min_spring_temp.values()),
    "Max Temp": list(max_spring_temp.values()),
    "Mean Temp": list(mean_spring_temp.values()),
    "Season": ["Spring"] * len(min_spring_temp)  # Add a tag
})

print(df_spring_summary)

   Year  Min Temp   Max Temp  Mean Temp  Season
0  1950  4.476717  12.600250   8.617391  Spring
1  1960  4.716065  12.575305   8.704000  Spring
2  1970  4.378891  12.068076   8.263473  Spring
3  1980  4.673511  12.400143   8.606750  Spring
4  1990  5.568946  13.458620   9.587453  Spring
5  2000  5.700413  13.559217   9.683687  Spring
6  2010  5.521982  13.618994   9.611927  Spring
7  2020  5.711152  14.384739  10.057565  Spring


In [31]:
min_summer_temp = {
    "1950":df_summer['1950']['temperature_2m_min'].mean(),
    "1960":df_summer['1960']['temperature_2m_min'].mean(),
    "1970":df_summer['1970']['temperature_2m_min'].mean(),
    "1980":df_summer['1980']['temperature_2m_min'].mean(),
    "1990":df_summer['1990']['temperature_2m_min'].mean(),
    "2000":df_summer['2000']['temperature_2m_min'].mean(),
    "2010":df_summer['2010']['temperature_2m_min'].mean(),
    "2020":df_summer['2020']['temperature_2m_min'].mean()
}


max_summer_temp = {
    "1950":df_summer['1950']['temperature_2m_max'].mean(),
    "1960":df_summer['1960']['temperature_2m_max'].mean(),
    "1970":df_summer['1970']['temperature_2m_max'].mean(),
    "1980":df_summer['1980']['temperature_2m_max'].mean(),
    "1990":df_summer['1990']['temperature_2m_max'].mean(),
    "2000":df_summer['2000']['temperature_2m_max'].mean(),
    "2010":df_summer['2010']['temperature_2m_max'].mean(),
    "2020":df_summer['2020']['temperature_2m_max'].mean()
}

mean_summer_temp = {
    "1950":df_summer['1950']['temperature_2m_mean'].mean(),
    "1960":df_summer['1960']['temperature_2m_mean'].mean(),
    "1970":df_summer['1970']['temperature_2m_mean'].mean(),
    "1980":df_summer['1980']['temperature_2m_mean'].mean(),
    "1990":df_summer['1990']['temperature_2m_mean'].mean(),
    "2000":df_summer['2000']['temperature_2m_mean'].mean(),
    "2010":df_summer['2010']['temperature_2m_mean'].mean(),
    "2020":df_summer['2020']['temperature_2m_mean'].mean()
}

# Combine into a DataFrame
df_summer_summary = pd.DataFrame({
    "Year": list(min_summer_temp.keys()),
    "Min Temp": list(min_summer_temp.values()),
    "Max Temp": list(max_summer_temp.values()),
    "Mean Temp": list(mean_summer_temp.values()),
    "Season": ["Summer"] * len(min_summer_temp)  # Add a tag
})

print(df_summer_summary)

   Year   Min Temp   Max Temp  Mean Temp  Season
0  1950  12.136284  20.369272  16.364819  Summer
1  1960  11.715848  19.905413  15.940916  Summer
2  1970  11.878619  20.006933  16.075304  Summer
3  1980  12.054217  20.092859  16.186602  Summer
4  1990  12.524327  20.864000  16.828297  Summer
5  2000  12.807967  20.764326  16.919937  Summer
6  2010  12.909699  21.107744  17.101273  Summer
7  2020  13.149739  22.260174  17.812532  Summer


In [32]:
min_autumn_temp = {
    "1950":df_autumn['1950']['temperature_2m_min'].mean(),
    "1960":df_autumn['1960']['temperature_2m_min'].mean(),
    "1970":df_autumn['1970']['temperature_2m_min'].mean(),
    "1980":df_autumn['1980']['temperature_2m_min'].mean(),
    "1990":df_autumn['1990']['temperature_2m_min'].mean(),
    "2000":df_autumn['2000']['temperature_2m_min'].mean(),
    "2010":df_autumn['2010']['temperature_2m_min'].mean(),
    "2020":df_autumn['2020']['temperature_2m_min'].mean()
}

max_autumn_temp = {
    "1950":df_autumn['1950']['temperature_2m_max'].mean(),
    "1960":df_autumn['1960']['temperature_2m_max'].mean(),
    "1970":df_autumn['1970']['temperature_2m_max'].mean(),
    "1980":df_autumn['1980']['temperature_2m_max'].mean(),
    "1990":df_autumn['1990']['temperature_2m_max'].mean(),
    "2000":df_autumn['2000']['temperature_2m_max'].mean(),
    "2010":df_autumn['2010']['temperature_2m_max'].mean(),
    "2020":df_autumn['2020']['temperature_2m_max'].mean()
}

mean_autumn_temp = {
    "1950":df_autumn['1950']['temperature_2m_mean'].mean(),
    "1960":df_autumn['1960']['temperature_2m_mean'].mean(),
    "1970":df_autumn['1970']['temperature_2m_mean'].mean(),
    "1980":df_autumn['1980']['temperature_2m_mean'].mean(),
    "1990":df_autumn['1990']['temperature_2m_mean'].mean(),
    "2000":df_autumn['2000']['temperature_2m_mean'].mean(),
    "2010":df_autumn['2010']['temperature_2m_mean'].mean(),
    "2020":df_autumn['2020']['temperature_2m_mean'].mean()
}

# Combine into a DataFrame
df_autumn_summary = pd.DataFrame({
    "Year": list(min_autumn_temp.keys()),
    "Min Temp": list(min_autumn_temp.values()),
    "Max Temp": list(max_autumn_temp.values()),
    "Mean Temp": list(mean_autumn_temp.values()),
    "Season": ["Autumn"] * len(min_autumn_temp)  # Add a tag
})

print(df_autumn_summary)

   Year  Min Temp   Max Temp  Mean Temp  Season
0  1950  7.324989  13.935703  10.636233  Autumn
1  1960  7.560704  13.998066  10.750520  Autumn
2  1970  7.412352  13.848341  10.645300  Autumn
3  1980  7.768671  14.074330  10.945790  Autumn
4  1990  7.809714  14.115484  10.936221  Autumn
5  2000  8.366967  14.765153  11.629696  Autumn
6  2010  8.332654  14.788972  11.585562  Autumn
7  2020  8.987204  16.046326  12.480954  Autumn


In [33]:
df_summary = pd.concat(
    [df_winter_summary,df_spring_summary,df_summer_summary,df_autumn_summary]
)
df_summary

In [34]:

long_df = px.data.medals_long()

fig = px.bar(long_df, x="nation", y="count", color="medal", title="Long-Form Input")
fig.show()